In [1]:
import numpy as np
import random as rd
import pandas as pd

In [2]:
train = pd.read_csv("./train.csv")
data = train.drop("species", axis=1).to_numpy()
label = train["species"].to_numpy()
data = (data - np.min(data, axis=0)) / (np.max(data, axis=0)-np.min(data, axis=0))

In [3]:
def mulnorm(x, u, c, k):
    if np.linalg.det(c) == 0:
        c = c+0.01*np.eye(k)
    d = pow(pow((2*np.pi), k)*np.linalg.det(c), -0.5)
    e = -0.5*np.dot(np.dot(x-u, np.linalg.inv(c)), x-u)
    return d*np.exp(e) + 1e-9


def PM(x, A, U, C):
    k = A.shape[0]
    pm = np.zeros(k)
    for i in range(k):
        pm[i] = A[i]*mulnorm(x, U[i], C[i], k)
    pm = pm / np.sum(pm)
    return pm


In [4]:
[K, N, D] = [3, data.shape[0], data.shape[1]]
[A, U, C] = [np.ones(K)/K, np.ones((K, D))/K, np.array([np.eye(D)*rd.random() for i in range(K)])]
R = np.ones((N, K))/K

In [5]:
for epoch in range(100):
    for x in range(N):
        R[x] = PM(data[x], A, U, C)
    for k in range(K):
        U[k] = np.average(data, weights=R[:, k], axis=0)
        C[k] = np.average(np.matmul((data - U[k]).reshape([-1, D, 1]), (data - U[k]).reshape([-1, 1, D])), weights=R[:, k], axis=0)
        A[k] = np.sum(R[:, k]) / N

In [6]:
def Evaluate(predict, label):
    rr, rn, nr, nn = 0, 0, 0, 0
    for i in range(len(label)):
        for j in range(i+1, len(label)):
            if label[i] == label[j] and predict[i] == predict[j]:
                rr = rr + 1
            if label[i] == label[j] and predict[i] != predict[j]:
                rn = rn + 1
            if label[i] != label[j] and predict[i] == predict[j]:
                nr = nr + 1
            if label[i] != label[j] and predict[i] != predict[j]:
                nn = nn + 1
    print(rr / (rr+rn+nr), pow(rr*rr/(rr+rn) / (rr+nr), 0.5), (rr+nn) / (rr+rn+nr+nn))

In [7]:
Evaluate(R.argmax(axis=1), label)

0.5780995475113122 0.7418544414651049 0.7914093959731544
